In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import os 
import urllib

In [7]:
import re

In [31]:
urllib.request.urlretrieve('http://b.hiphotos.baidu.com/image/h%3D300/sign=92afee66fd36afc3110c39658318eb85/908fa0ec08fa513db777cf78376d55fbb3fbd9b3.jpg','sunflower.jpg')

('sunflower.jpg', <http.client.HTTPMessage at 0x116100898>)

![](sunflower.jpg)

In [3]:
!ls &&pwd 
os.path.isfile("/Applications/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/")

Decorators.ipynb       Multithreadings.ipynb  sunflower.jpg
FileManipulation.ipynb Python3Skeleton.ipynb
Generators.ipynb       pikaHello.ipynb
/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge


False

In [8]:
pattern=re.compile(r"\.\w{5}")
for root, dirs, files in os.walk("/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/"):
    for file in files:
        #if pattern.findall(file)[0] == '.ipynb':
        print( file)

sunflower.jpg
Decorators.ipynb
FileManipulation.ipynb
Generators.ipynb
Python3Skeleton.ipynb
Multithreadings.ipynb
pikaHello.ipynb
Multithreadings-checkpoint.ipynb
Python3Skeleton-checkpoint.ipynb
Decorators-checkpoint.ipynb
FileManipulation-checkpoint.ipynb
Generators-checkpoint.ipynb


In [9]:
from os import listdir
from os.path import isfile, join
mypath = "/Users/wangmiao/Playground/GH/IPython_training/basic/WiderKnowledge/"
[f for f in listdir(mypath) if isfile(join(mypath, f))]

['sunflower.jpg',
 'Decorators.ipynb',
 'FileManipulation.ipynb',
 'Generators.ipynb',
 'Python3Skeleton.ipynb',
 'Multithreadings.ipynb',
 'pikaHello.ipynb']

In [13]:
pattern.findall("Decorators.ipynb")[0]

'.ipynb'

In [10]:
class Card:
    def __init__(self):
        self.suit = ["Hearts","Diamonds","Clubs","Spades"]
        self.value = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
        self.cards = [] 

        for x in self.value:
            y = x + " of"
            for z in self.suit :
                aa = y + " " + z
                self.cards.append(aa)

    def __repr__(self): 
        return str(self.cards) 

cards1 = Card()

In [11]:
cards1.cards

['A of Hearts',
 'A of Diamonds',
 'A of Clubs',
 'A of Spades',
 '2 of Hearts',
 '2 of Diamonds',
 '2 of Clubs',
 '2 of Spades',
 '3 of Hearts',
 '3 of Diamonds',
 '3 of Clubs',
 '3 of Spades',
 '4 of Hearts',
 '4 of Diamonds',
 '4 of Clubs',
 '4 of Spades',
 '5 of Hearts',
 '5 of Diamonds',
 '5 of Clubs',
 '5 of Spades',
 '6 of Hearts',
 '6 of Diamonds',
 '6 of Clubs',
 '6 of Spades',
 '7 of Hearts',
 '7 of Diamonds',
 '7 of Clubs',
 '7 of Spades',
 '8 of Hearts',
 '8 of Diamonds',
 '8 of Clubs',
 '8 of Spades',
 '9 of Hearts',
 '9 of Diamonds',
 '9 of Clubs',
 '9 of Spades',
 '10 of Hearts',
 '10 of Diamonds',
 '10 of Clubs',
 '10 of Spades',
 'J of Hearts',
 'J of Diamonds',
 'J of Clubs',
 'J of Spades',
 'Q of Hearts',
 'Q of Diamonds',
 'Q of Clubs',
 'Q of Spades',
 'K of Hearts',
 'K of Diamonds',
 'K of Clubs',
 'K of Spades']

In [13]:
ToEndofDays = 6388
#find today's date
import datetime
from datetime import date
EOD = "2037-01-04"
EOD = datetime.date(2037, 1, 4) 
print (EOD)
#d=datetime.datetime.today()
d=datetime.date.today()
# td = str(d(0-9))
# print (td)
delta = EOD - d
print ("Today is", d)
#print (ToEndofDays) 
#print ("I have until", EOD, "when I die"")
print(ToEndofDays, "days until the end!")
print (delta)

2037-01-04
Today is 2019-07-16
6388 days until the end!
6382 days, 0:00:00


In [14]:
s = "http://asaa.com"

In [19]:
import requests, bs4

url = 'https://www.basketball-reference.com/players/a/abrinal01.html'
res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('#per_game')

table = soup.find("table", { "id" : "per_game" })
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    
    print(row)

[]
['23', 'OKC', 'NBA', 'SG', '68', '6', '15.5', '2.0', '5.0', '.393', '1.4', '3.6', '.381', '0.6', '1.4', '.426', '.531', '0.6', '0.7', '.898', '0.3', '1.0', '1.3', '0.6', '0.5', '0.1', '0.5', '1.7', '6.0']
['24', 'OKC', 'NBA', 'SG', '75', '8', '15.1', '1.5', '3.9', '.395', '1.1', '2.9', '.380', '0.4', '0.9', '.443', '.540', '0.5', '0.6', '.848', '0.3', '1.2', '1.5', '0.4', '0.5', '0.1', '0.3', '1.7', '4.7']
['25', 'OKC', 'NBA', 'SG', '31', '2', '19.0', '1.8', '5.1', '.357', '1.3', '4.1', '.323', '0.5', '1.0', '.500', '.487', '0.4', '0.4', '.923', '0.2', '1.4', '1.5', '0.6', '0.5', '0.2', '0.5', '1.7', '5.3']
['', '', 'NBA', '', '174', '16', '16.0', '1.8', '4.5', '.387', '1.3', '3.4', '.368', '0.5', '1.1', '.443', '.525', '0.5', '0.6', '.880', '0.3', '1.1', '1.4', '0.5', '0.5', '0.1', '0.4', '1.7', '5.3']


In [20]:
import requests, bs4

url = 'https://www.basketball-reference.com/players/a/abrinal01.html'
res = requests.get(url)
res.raise_for_status()

soup = bs4.BeautifulSoup(res.text, 'html.parser')
elems = soup.select('#per_game')

table = soup.find("table", { "id" : "per_game" })
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td') + tr.find_all('th')
    row = [i.text for i in td]
    print(row)

['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
['23', 'OKC', 'NBA', 'SG', '68', '6', '15.5', '2.0', '5.0', '.393', '1.4', '3.6', '.381', '0.6', '1.4', '.426', '.531', '0.6', '0.7', '.898', '0.3', '1.0', '1.3', '0.6', '0.5', '0.1', '0.5', '1.7', '6.0', '2016-17']
['24', 'OKC', 'NBA', 'SG', '75', '8', '15.1', '1.5', '3.9', '.395', '1.1', '2.9', '.380', '0.4', '0.9', '.443', '.540', '0.5', '0.6', '.848', '0.3', '1.2', '1.5', '0.4', '0.5', '0.1', '0.3', '1.7', '4.7', '2017-18']
['25', 'OKC', 'NBA', 'SG', '31', '2', '19.0', '1.8', '5.1', '.357', '1.3', '4.1', '.323', '0.5', '1.0', '.500', '.487', '0.4', '0.4', '.923', '0.2', '1.4', '1.5', '0.6', '0.5', '0.2', '0.5', '1.7', '5.3', '2018-19']
['', '', 'NBA', '', '174', '16', '16.0', '1.8', '4.5', '.387', '1.3', '3.4', '.368', '0.5', '1.1', '.443', '.525', '0.5', '0.6', '.880', '0.3', '1.1

In [33]:
def througput():
    print("---------------menghitung througput Tower-------------------")
    print ("====Plih band====")

    print("[1]. 5 MHz, 25 RB")
    print("[2]. 10 MHz, 50 RB")
    print("[3]. 3 MHz, 75 RB")
    print("[4].  MHz, 100 RB")
    band =  int(input ("masukan pilihan (RB bandwith) : "))

    print ("====Plih Modulasi====")
    print("[1]. 64QAM")
    print("[2]. 16QAM")
    print("[3]. QPSK")
    mod =  int(input ("masukan pilihan (bits/symbol) : "))


    print ("====Plih Antena====")
    print("[1]. SISO")
    print("[2]. 2x2 MIMO")
    print("[3]. 4x4 MIMO")
    anten = int(input ("masukan pilihan (Pengali Antena: "))
    
    bandchoice = ["5 MHz, 25 RB", "10 MHz, 50 RB", "3 MHz, 75 RB", "MHz, 100 RB"]
    modchoice = ["64QAM", "16QAM", "QPSK"]
    antenchoice = ["SISO", "2x2 MIMO", "4x4 MIMO"]


    return ("band = {} mod = {}  anten= {}".format(bandchoice[band-1], modchoice[mod-1], antenchoice[anten-1]))

In [34]:
througput()

---------------menghitung througput Tower-------------------
====Plih band====
[1]. 5 MHz, 25 RB
[2]. 10 MHz, 50 RB
[3]. 3 MHz, 75 RB
[4].  MHz, 100 RB
masukan pilihan (RB bandwith) : 1
====Plih Modulasi====
[1]. 64QAM
[2]. 16QAM
[3]. QPSK
masukan pilihan (bits/symbol) : 1
====Plih Antena====
[1]. SISO
[2]. 2x2 MIMO
[3]. 4x4 MIMO
masukan pilihan (Pengali Antena: 1


'band = 5 MHz, 25 RB mod = 64QAM  anten= SISO'